In [1]:
import numpy as np
import pandas as pd
import jittor as jt
import os
from os.path import join, getsize

from jittor.dataset import Dataset
from PIL import Image
from dataset import Tiny_vid

from model import DetNet, train, test, total_acc, total_num, main


[i 0419 20:06:57.385932 88 log.cc:351] Load log_sync: 1
[i 0419 20:06:57.482737 88 compiler.py:951] Jittor(1.3.2.5) src: /opt/miniconda/envs/dhy/lib/python3.9/site-packages/jittor
[i 0419 20:06:57.490623 88 compiler.py:952] g++ at /usr/bin/g++(9.3.0)
[i 0419 20:06:57.492340 88 compiler.py:953] cache_path: /home/dhy/.cache/jittor/jt1.3.2/g++9.3.0/py3.9.7/Linux-3.10.0-1xeb/IntelRXeonRGolx19/default
[i 0419 20:06:57.505703 88 install_cuda.py:51] cuda_driver_version: [11, 4]
[i 0419 20:06:57.512621 88 __init__.py:411] Found /home/dhy/.cache/jittor/jtcuda/cuda11.2_cudnn8_linux/bin/nvcc(11.2.152) at /home/dhy/.cache/jittor/jtcuda/cuda11.2_cudnn8_linux/bin/nvcc.
[i 0419 20:06:57.575119 88 __init__.py:411] Found gdb(9.2) at /usr/bin/gdb.
[i 0419 20:06:57.582104 88 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0419 20:06:57.712520 88 compiler.py:1006] cuda key:cu11.2.152_sm_86
[i 0419 20:06:57.921914 88 __init__.py:227] Total mem: 251.43GB, using 16 procs for compiling.
[i 04

In [2]:
from jittor import nn
import jittor.transform as transfrom
import numpy as np
from jittor.models.resnet import Resnet50, ResNet, Resnet34
from dataset import Tiny_vid
from argparse import ArgumentParser
from bbox import box_iou_batch
jt.misc.set_global_seed(425)

In [3]:
def bbox_to_rect(bbox, color):
    """Defined in :numref:`sec_bbox`"""
    # 将边界框(左上x,左上y,右下x,右下y)格式转换成matplotlib格式：
    # ((左上x,左上y),宽,高)
    return plt.Rectangle(
        xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0], height=bbox[3]-bbox[1],
        fill=False, edgecolor=color, linewidth=2)
    
def show_bboxes(axes, bboxes, labels=None, colors=None):
    def _make_list(obj, default_values=None):
        if obj is None:
            obj = default_values
        elif not isinstance(obj, (list, tuple)):
            obj = [obj]
        return obj

    labels = _make_list(labels)
    colors = _make_list(colors, ['b', 'g', 'r', 'm', 'c'])

    for i, bbox in enumerate(bboxes):
        color = colors[i % len(colors)]
        rect = bbox_to_rect(bbox, color)
        axes.add_patch(rect)
        if labels and len(labels) > i:
            text_color = 'k' if color == 'w' else 'w'
            axes.text(rect.xy[0], rect.xy[1], labels[i], va='center',
                      ha='center', fontsize=9, color=text_color, bbox=dict(facecolor=color, lw=0))
            
def show_bbox(outboxes, outclass, input, label,id=0):
    cdict = {'bird':0,'car':1,'dog':2,'lizard':3,'turtle':4}
    clist = list(cdict.keys())
    pred,_ = jt.argmax(outclass.data[id,:], dim=0)
    outlabel = clist[int(pred)]
    outboxes *= 128
    outbox = np.floor(outboxes)[id,:]
    realbox = np.array(label[1][id,:]*128).astype(np.int64)
    reallabel = clist[int(label[id][0])]
    from PIL import Image
    img = jt.transform.to_pil_image(input[id,:].transpose(2,1,0))
    fig = plt.figure()
    axes = fig.add_axes([0,0,1,1])
    plt.imshow(img)
    # import pdb;pdb.set_trace()
    show_bboxes(axes, [realbox], labels=reallabel, colors='black')
    show_bboxes(axes, [outbox], labels=outlabel, colors='blue')
    plt.show()
    
# show_bbox(outboxes, outclass, inputs, targets)    
    

    

In [4]:
jt.flags.use_cuda = 1
batch_size = 16
learning_rate = 1e-2
momentum = 0.9
weight_decay = 1e-4
epochs = 20

[i 0419 20:07:01.394360 88 cuda_flags.cc:32] CUDA enabled.


In [5]:
import matplotlib.pyplot as plt
from IPython import display

In [6]:
my_transform = transfrom.Compose([
    transfrom.ToTensor()
    ])

train_loader = Tiny_vid(train=True, transform=my_transform, aug=True).set_attrs(batch_size=batch_size, shuffle=True)
val_loader = Tiny_vid(train=False, transform=my_transform)
val_loader.set_attrs(batch_size=len(val_loader.ground_truth), shuffle=False)

model = DetNet()
optimizer = nn.SGD(model.preprocess.parameters()+model.backbone.parameters()+model.class_head.parameters(), learning_rate, momentum, weight_decay)
optimizer_box = nn.SGD(model.preprocess.parameters()+model.bbox_head.parameters(), learning_rate, momentum, weight_decay)
for epoch in range(epochs):
    train(model, train_loader, optimizer, optimizer_box, epoch)
    outclass, outboxes, inputs, targets = test(model, val_loader, epoch)
    if epoch%5 == 0:
        display.clear_output(wait=True)
    show_bbox(outboxes, outclass, inputs, targets) 
    # break


dataset img len: 1500
dataset label len: 1500
dataset img len: 150
dataset label len: 150


KeyboardInterrupt: 

In [ ]:
outclass[0,:]
pred,_ = jt.argmax(outclass.data[0,:], dim=0)
int(pred)


In [ ]:
outboxes[0,:]

In [ ]:
cdict = {'bird':0,'car':1,'dog':2,'lizard':3,'turtle':4}
list(cdict.keys())

In [ ]:
targets[1][0,:]
a = np.array(targets[1][0,:]*128).astype(np.int64)
bbox_to_rect(a,'b')
# realbox = np.floor(targets[0][1]*128)